if you have questions on this assignment, please send me (Achille Mascia) an email achille.mascia@columbia.edu or come see me during my office hours.

<h1>Scraping Fidelity.com</h1>
In this assignment, you will scrape data from fidelity.com. The goal of the exercise is to get the latest sector performance data from the US markets, and to get the total market capitalization for each sector. 

The end result is to write a function: <i>get_us_sector_performance()</i> that will return a list of tuples. Each tuple should correspond to a sector and should contain the following data:
<li>the sector name
<li>the amount the sector has moved
<li>the market capitalization of the sector
<li>the market weight of the sector
<li>a link to the fidelity page for that sector

<p>
The data should be sorted by decreasing order of market weight. I.e., the sector with the highest weight should be in the first tuple, etc.

<h3>Process</h3>
<li>Get a list of sectors and the links to the sector detail pages from the url (see function)
<li>Loop through the list and call the function <i>get_sector_change_and_market_cap(sector_page_link)</i> for each sector
<li>Accumulate the name, the change, the capitalization, the weight and the link for each sector in output_list (see function)
<li>Sort the list by market weight

<b>Notes:</b>
<li>Note that the market weight is a string with a % sign at the back. You will need to get rid of the % and convert the string into a float before you can sort it
<li>Your starting data is the url listed below. You need to extract all data, including links to the sector pages, from the page at this url
<li>To sort a list of tuples by an arbitrary element, use the example at the bottom of this notebook

In [8]:
def get_us_sector_performance():
    output_list = list()
    url = "https://eresearch.fidelity.com/eresearch/goto/markets_sectors/landing.jhtml"

    import requests
    from bs4 import BeautifulSoup
    
    try:
        response = requests.get(url)
        if response.status_code == 200:
            page_data_soup = BeautifulSoup(response.content, 'lxml')
            
            table = page_data_soup.find('div', class_='performance-section').find('table').find_all('div', class_='heading')
            
            sector_list = [x.get_text() for x in table]
            link_list = ['https://eresearch.fidelity.com/' + x.find('a').get('href') for x in table]
            
            change_list = []
            market_list = []
            market_weight_list = []
                
            
            for link in link_list:
                sector_change,sector_market_cap,sector_market_weight = get_sector_change_and_market_cap(link)
                change_list.append(sector_change)
                market_list.append(sector_market_cap)
                market_weight_list.append(sector_market_weight)
             
            output_list = list(zip(sector_list, change_list, market_list, market_weight_list, link_list))

    except:
        return 'None'
    
    output_list.sort(key = lambda x: x[3], reverse = True)

    return output_list
    

In [9]:
def get_sector_change_and_market_cap(sector_page_link):
    import requests
    from bs4 import BeautifulSoup
    
    try:
        response = requests.get(sector_page_link)
        if response.status_code == 200:
            page_data_soup = BeautifulSoup(response.content, 'lxml')
            
            table = page_data_soup.find('div', class_ = 'tab-content-wrapper').find('table').find('tbody')

            sector_change = float(table.find('span').get_text().strip('%').strip('+'))

            sector_market_cap = table.find_all('span')[2].get_text()

            sector_market_weight = float(table.find('td', class_= 'left-spcng').find('span').get_text().strip('%'))
    except:
        return 'None'
        
    return sector_change,sector_market_cap,sector_market_weight

In [10]:
#Test get_sector_change_and_market_cap()
link = "https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=25"
print(get_sector_change_and_market_cap(link))

(-1.9, '$4.78T', 10.0)


In [11]:
#Test get_us_sector_performance()
get_us_sector_performance()

[('Information Technology',
  -2.01,
  '$7.25T',
  20.83,
  'https://eresearch.fidelity.com//eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=45'),
 ('Health Care',
  -1.56,
  '$5.68T',
  15.14,
  'https://eresearch.fidelity.com//eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=35'),
 ('Financials',
  -1.49,
  '$7.15T',
  13.43,
  'https://eresearch.fidelity.com//eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=40'),
 ('Communication Services',
  -1.93,
  '$4.65T',
  10.0,
  'https://eresearch.fidelity.com//eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=50'),
 ('Consumer Discretionary',
  -1.92,
  '$4.78T',
  10.0,
  'https://eresearch.fidelity.com//eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=25'),
 ('Industrials',
  -1.89,
  '$4.04T',
  9.65,
  'https://eresearch.fidelity.com//eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector